# Cellpose Installation and Testing – Binder‑Friendly Legacy Version

This notebook installs **Cellpose 3.2.3** (the last release with the classic `models.Cellpose` API) so it works out‑of‑the‑box on [mybinder.org](https://mybinder.org).

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/your-username/your-repo/HEAD?filepath=cellpose_complete.ipynb)

## What This Notebook Does
✅ Installs legacy Cellpose 3.x  
✅ Creates synthetic test data  
✅ Runs segmentation successfully  
✅ Visualises results  
✅ Provides working code snippets you can reuse

## Step 1 – Environment Setup

In [ ]:
import sys, os, subprocess, time

print(f"Python version: {sys.version}")
print(f"Working directory: {os.getcwd()}")
print(f"Platform: {sys.platform}")

def run_cmd(cmd, timeout=300):
    """Utility: run shell command and return (ok, stdout, stderr)"""
    try:
        res = subprocess.run(cmd, shell=True, capture_output=True, text=True, timeout=timeout)
        return res.returncode == 0, res.stdout.strip(), res.stderr.strip()
    except subprocess.TimeoutExpired:
        return False, "", f"Command timed out after {timeout}s"

## Step 2 – Install Cellpose 3.x and Dependencies

In [ ]:
print("🔧 Installing Cellpose 3.2.3 and core libraries … (2‑3 min on Binder)")

packages = [
    "numpy",
    "scipy",
    "matplotlib",
    "scikit-image",
    "opencv-python-headless",
    "tifffile",
    "torch==2.2.2",
    "torchvision==0.17.2",
    "cellpose==3.2.3"  # 👈 pin legacy version
]

ok, fail = [], []
for i, pkg in enumerate(packages, 1):
    print(f"[{i}/{len(packages)}] {pkg} …", end=" ")
    success, _, err = run_cmd(f"pip install --quiet {pkg}")
    if success:
        print("✅")
        ok.append(pkg)
    else:
        print("❌")
        print(err[:200])
        fail.append(pkg)

installation_successful = len(fail) == 0
print(f"\n📊 Installed: {len(ok)}  |  Failed: {len(fail)}")

## Step 3 – Verify Cellpose Import & Model Init

In [ ]:
print("🔍 Importing Cellpose …")
try:
    import cellpose, numpy as np, matplotlib.pyplot as plt, tifffile
    from cellpose import models, utils
    print("✅ Cellpose imported – v", cellpose.__version__)
    model = models.Cellpose(gpu=False, model_type='cyto')
    print("✅ Model initialised (cyto)")
    cellpose_working = True
except Exception as e:
    print("❌ Cellpose failed:", e)
    cellpose_working = False

import matplotlib
if cellpose_working:
    matplotlib.use('Agg'); plt.ioff()

## Step 4 – Create Synthetic Test Data

In [ ]:
from scipy import ndimage
if cellpose_working:
    print("🔬 Generating synthetic image …")
    np.random.seed(42)

    def make_cells(size=(512,512), n=12):
        img = np.zeros(size, np.float32)
        Y, X = np.ogrid[:size[0], :size[1]]
        for _ in range(n):
            cy, cx = np.random.randint(50, size[0]-50), np.random.randint(50, size[1]-50)
            r = np.random.randint(18,35)
            noise = np.random.normal(0,3,size)
            mask = np.sqrt((X-cx)**2 + (Y-cy)**2) <= r + 0.3*noise
            img[mask] = np.random.uniform(160,240)
        img += np.random.normal(20,8,size)
        img = ndimage.gaussian_filter(img,1.5)
        return np.clip(img,0,255).astype(np.uint8)

    test_img = make_cells(); tifffile.imwrite('test_cells.tif', test_img)
    print("✅ test_cells.tif saved")
    plt.figure(figsize=(6,5)); plt.imshow(test_img,cmap='gray'); plt.axis('off');
    plt.tight_layout(); plt.savefig('test_cells_preview.png',dpi=120)
    test_data_created = True
else:
    test_data_created = False

## Step 5 – Segment with Cellpose

In [ ]:
if cellpose_working and test_data_created:
    print("🎯 Segmenting …")
    img = tifffile.imread('test_cells.tif')
    t0 = time.time()
    masks, flows, styles, diams = model.eval(img, diameter=None, channels=[0,0])
    dt = time.time()-t0
    n_cells = len(np.unique(masks))-1
    print(f"✅ Done in {dt:.1f}s  |  Cells: {n_cells}")
    segmentation_success = True
else:
    segmentation_success = False

## Step 6 – Visualise & Save Outputs

In [ ]:
if segmentation_success:
    print("📊 Plotting …")
    from skimage import measure
    fig,ax = plt.subplots(1,3,figsize=(15,5))
    ax[0].imshow(img,cmap='gray'); ax[0].set_title('Original'); ax[0].axis('off')
    ax[1].imshow(masks,cmap='nipy_spectral'); ax[1].set_title(f'Masks ({n_cells})'); ax[1].axis('off')
    ax[2].imshow(img,cmap='gray'); ax[2].contour(masks,[0.5],colors='r'); ax[2].set_title('Overlay'); ax[2].axis('off')
    plt.tight_layout(); plt.savefig('cellpose_results.png',dpi=150)

    tifffile.imwrite('segmentation_masks.tif', masks.astype(np.uint16))
    outlines = utils.masks_to_outlines(masks); tifffile.imwrite('cell_outlines.tif', outlines.astype(np.uint8)*255)

    props = measure.regionprops(masks,img); areas=[p.area for p in props]
    print(f"🔢 Cell area mean±sd: {np.mean(areas):.1f} ± {np.std(areas):.1f} px²")
    visualization_success = True
else:
    visualization_success = False

## Step 7 – Practical Legacy Cellpose Snippets

In [ ]:
print("📚 BASIC WORKFLOW (legacy API)")
print("="*60)
print(
"""
from cellpose import models
import tifffile, numpy as np

# 1. Load image
img = tifffile.imread('your_image.tif')

# 2. Init model (CPU)
model = models.Cellpose(gpu=False, model_type='cyto')   # or 'nuclei', 'cyto2'

# 3. Run segmentation
masks, flows, styles, diams = model.eval(
    img,
    diameter=None,          # auto
    channels=[0,0],         # [cytoplasm, nucleus] – use [0,0] for grayscale
    flow_threshold=0.4,
    cellprob_threshold=0.0
)

# 4. Save
tifffile.imwrite('masks.tif', masks.astype(np.uint16))
"""
)
if segmentation_success:
    print(f"\n✅ Notebook run detected {n_cells} cells and saved output files.")

## Step 8 – Final Report

In [ ]:
print("🎉 FINAL STATUS\n"+"="*40)
overall = all([installation_successful, cellpose_working, test_data_created, segmentation_success, visualization_success])
print("All steps succeeded ✅" if overall else "Some steps failed ⚠️")
for f in [
    'test_cells.tif','test_cells_preview.png',
    'cellpose_results.png','segmentation_masks.tif','cell_outlines.tif'
]:
    print("✔" if os.path.exists(f) else "✘", f)